In [1]:
import requests
import pandas as pd

import netCDF4 as nc#not directly used but needs to be imported for some nc4 files manipulations, use for nc files
from netCDF4 import Dataset
import numpy as np
import numpy.ma as ma

In [2]:
# register information from csv file
all_urls = pd.read_csv(r'C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool\outputs\NEX-GDDP-CMIP6\gddp-cmip6-thredds-fileserver.csv')

In [3]:
### make all elements of the csv into a readable list

# transpose csv
temp_list = all_urls[[' fileUrl']].T
temp_list=temp_list.values.tolist()
temp_list=temp_list[0]
url_list=[s.replace(' ', '') for s in temp_list]

In [ ]:
for file in url_list:
    # find the name of the file
    index_before_name=file.rfind('/') # returns the highest index where the last character '/' was found, which is just before the name of the file    
    f_name = file[index_before_name+1:len(file)-3] # return the name pf the file as a string
    #r = requests.get(requests.get(file.url)) # return the url were data need to be downloaded
    r = requests.get(file) # return the url were data need to be downloaded
    # download data in the servor for datasets
    with open(f'//COWI.net/projects/A245000/A248363/CRVA/Datasets/NEX-GDDP-CMIP6/{f_name}.nc', 'wb') as f:
        f.write(r.content)

In [ ]:
## read data, _FillValue should be NaN

In [ ]:
solar_1950=Dataset(r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6\hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.nc')

In [ ]:
hurs=solar_1950.variables['hurs']

In [ ]:
hurs

In [ ]:
hurs._FillValue

In [ ]:
solar_dataframe = np.ma.getdata(solar_1950.variables['hurs']).data

In [ ]:
solar_dataframe

In [ ]:
len(solar_dataframe[solar_dataframe!=1.e+20])

In [ ]:
len(solar_dataframe[solar_dataframe==1.e+20])

In [ ]:
type(solar_dataframe)

In [ ]:
solar_dataframe.size

In [ ]:
solar_dataframe.size-len(solar_dataframe[solar_dataframe!=1.e+20])-len(solar_dataframe[solar_dataframe==1.e+20])